# Goals
+ summaries
+ lumen notices
    + lumen notices per day
+ twitter users
    + twitter users by CS_oldest_archived (should be all completed --3)
    + twitter users by date added
+ twitter statuses
    + count by record_created_at
    + by user
        + indicator before lumen notice served
    + user | notice time | tweets per day before (1day) | tweets per day after (1day)
+ twitter rate_state by checkin due
    
    
+ pivoted tables
    + date by
        + lumen notices
        + twitter user added
        + twitter statuses collected

In [1]:
import os, inspect, json
import pandas as pd
BASE_DIR = os.path.join(os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))), "..")

ENV = 'development'

if ENV == 'production':
    pass #open SSH tunnel to production

In [2]:
with open(os.path.join(BASE_DIR, "config") + "/{env}.json".format(env=ENV), "r") as config:
    DBCONFIG = json.loads(config.read())

### LOAD SQLALCHEMY
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
db_engine = create_engine("mysql://{user}:{password}@{host}/{database}".format(
    host=DBCONFIG['host'],
    user=DBCONFIG['user'],
    password=DBCONFIG['password'],
    database=DBCONFIG['database']))


In [3]:
lum_not_sql = '''SELECT date(date_received) as dr, count(date(date_received))
FROM civilservant_development.lumen_notices
GROUP by dr
'''
lum_not_df = pd.read_sql(lum_not_sql, db_engine, parse_dates=['dr'])

lum_not_df

,dr,count(date(date_received))
0,2018-09-06,18
1,2018-09-07,223
2,2018-09-08,91
3,2018-09-10,96
4,2018-09-11,4


In [4]:
twit_user_processed = '''-- how many in process
SELECT CS_oldest_tweets_archived, count(CS_oldest_tweets_archived) from twitter_users
	group by CS_oldest_tweets_archived
'''
twit_user_processed_df = pd.read_sql(twit_user_processed, db_engine)
twit_user_processed_df

,CS_oldest_tweets_archived,count(CS_oldest_tweets_archived)
0,1,925
1,2,370
2,3,570


In [5]:
twit_user_rec_created = '''select date(record_created_at) as drca, COUNT(date(record_created_at)) from twitter_users
GROUP by drca
'''
twit_user_rec_created_df = pd.read_sql(twit_user_rec_created, db_engine, parse_dates=['drca'])
twit_user_rec_created_df

,drca,COUNT(date(record_created_at))
0,2018-09-08,1865


In [6]:
lumen_twit = '''select notice_id, date_received, twitter_user_id from lumen_notice_to_twitter_user as lum_twit
left join (select id, date_received from lumen_notices) as lum_notice
on lum_twit.notice_id = lum_notice.id
'''
lumen_twit_df = pd.read_sql(lumen_twit, db_engine, parse_dates=['date_received'])
lumen_twit_df.head()

,notice_id,date_received,twitter_user_id
0,17213146,2018-09-06 18:09:00,1020128614107566081
1,17213146,2018-09-06 18:09:00,2305927712
2,17213146,2018-09-06 18:09:00,3475689025
3,17213146,2018-09-06 18:09:00,952487155405901824
4,17213146,2018-09-06 18:09:00,782051605839577088


In [7]:
twitid_complaints = lumen_twit_df.groupby('twitter_user_id').agg({'notice_id':len}).rename({'notice_id':'complaints_received'}, axis=1)

In [8]:
len(twitid_complaints[twitid_complaints['complaints_received']>1])/len(twitid_complaints)

0.05787781350482315

In [9]:
twitter_rate_sql = '''select max(reset_time), user_id from twitter_ratestate
group by user_id'''
twitter_rate_df = pd.read_sql(twitter_rate_sql, db_engine, parse_dates=['max(reset_time)'])
twitter_rate_df

,max(reset_time),user_id
0,2018-09-08 17:56:05,2833147639
1,2018-09-08 17:56:05,286504285
2,2018-09-08 17:56:05,3315303954
3,2018-09-08 17:56:05,411733308


In [10]:
before_after_tweets_sql = '''create TEMPORARY table notice_twit_user as
(
select n.date_received as notice_date, ltt.twitter_user_id as twitter_user_id from lumen_notices as n
join lumen_notice_to_twitter_user as ltt
on n.id = ltt.notice_id
)

select * from notice_twit_user

-- make this relation first and then later groupby
create TEMPORARY table notice_tweet as (
select notice_date, twitter_user_id, created_at as tweet_date
from notice_twit_user nt
join twitter_statuses ts
on nt.twitter_user_id = ts.user_id
)


select sum(before_tweet) as before_tweets, sum(after_tweet) as after_tweets, twitter_user_id 
from 
 (select tweet_date, 
		notice_date,
		case when tweet_date < notice_date then 1 else 0 end as before_tweet,
		case when tweet_date > notice_date then 1 else 0 end as after_tweet,
		twitter_user_id
	from notice_tweet ) as tweet_timing
group by twitter_user_id
'''